# Simple modal to predict if machine is working or not, after given number of hours it has been active

The training data should have labels of active working hours of similar machine and wether its working or not.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

Each row in the 2D tensor corresponds to a single data sample, and the columns correspond to the features of that sample. 

*Other parameters on makin a tensor*
- shape (or size()): Dimensions of the tensor.
- ndimension(): Number of dimensions (axes) the tensor has.
- numel(): Total number of elements in the tensor.
- is_cuda: Returns True if tensor is on a CUDA device (GPU).
- stride(): Returns the stride of the tensor (how far apart elements are).
- dim(): Number of dimensions (like ndimension()).
- flatten(): Flattens the tensor into a 1D tensor.
- view(): Reshapes the tensor to a new shape.
- transpose() or t(): Transposes a 2D tensor (swap rows and columns).
- clone(): Returns a copy of the tensor with the same content but different memory allocation.
- to(): Moves a tensor to another device or changes its data type.

In [18]:
# Dataset (feature: hours used)
# Labels: [0 = broken, 1 = working]
data = torch.tensor([
    [100],
    [0],
    [50],
    [900],
    [10],
    [3],
    [700],
                     [200], 
                     [1500], 
                     [800], 
                     [1200], 
                     [400]],
                   dtype=torch.float32)

labels = torch.tensor([
    0,
    1,
    1,
    0,
    1,
    1,
    0,
    1, 0, 1, 0, 1], dtype=torch.float32).view(-1, 1) 


In [19]:
# Checking the shapes of data and labels
print("Data:", data)
print("Labels:", labels)

Data: tensor([[ 100.],
        [   0.],
        [  50.],
        [ 900.],
        [  10.],
        [   3.],
        [ 700.],
        [ 200.],
        [1500.],
        [ 800.],
        [1200.],
        [ 400.]])
Labels: tensor([[0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.]])


# Defining a simple model with Linear Layer, Activation, and Layer Normalization

In [11]:
class MachineStatusModel(nn.Module):
    def __init__(self):
        super(MachineStatusModel, self).__init__()
        
        # the linear layer
        # Input size = 1 (hours used) and the output size = 1 (probability of working)
        self.linear = nn.Linear(1, 1)
        
        # Layer normalization and is applied after linear layer
        self.layer_norm = nn.LayerNorm(1)
        
        # This is Sigmoid activation function 
        #Is used for binary classification : (0 = broken, 1 = working)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Data flows through the network
        # Linear transformation: W*x + b
        x = self.linear(x)
        
        # Apply Layer Normalization
        x = self.layer_norm(x)
        
        # Apply Sigmoid to get the probability of machine working (between 0 and 1)
        x = self.sigmoid(x)
        
        return x

# Instantiating the model, loss function, and optimizer

nn.BCELoss(), BCELoss stands for Binary Cross-Entropy Loss, which is commonly used for binary classification tasks where the output is a probability value (i.e., between 0 and 1)

This loss function measures how well the model's predicted probabilities match the true binary labels (0 or 1). The output of a binary classifier is usually passed through a sigmoid activation function to map it to a probability between 0 and 1.

---

optim.SGD() , is the Stochastic Gradient Descent (SGD) optimizer. It's a commonly used optimization algorithm that updates model parameters based on the gradient of the loss function with respect to those parameters.

model.parameters() passes all the trainable parameters of the MachineStatusModel() to the optimizer.

lr=0.01 sets the learning rate for the optimizer. The learning rate determines the step size at each iteration while moving towards a minimum of the loss function. 
A small learning rate (e.g., 0.01) means the model will learn more slowly but can converge more precisely.

In [4]:
model = MachineStatusModel()
# Binary Cross-Entropy Loss (since this is a binary classification problem)
criterion = nn.BCELoss()
# Stochastic Gradient Descent (SGD) for optimization
optimizer = optim.SGD(model.parameters(), lr=0.01)


# Training the model with given data

In [21]:
epochs = 10000
for epoch in range(epochs):
    # Forward pass
    # compute the model output (probability of working)
    predictions = model(data)
    
    # finding the loss by passing predictions and expected labels
    loss = criterion(predictions, labels)
    

    # Backward pass: 
    # compute gradients
    # Clear previous gradients
    optimizer.zero_grad()  
    # New gradients
    loss.backward()  



    
    # Update weights using the optimizer
    optimizer.step()
    
    # Print the loss every 100 epochs to track progress
    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [100/10000], Loss: 0.6792
Epoch [200/10000], Loss: 0.6792
Epoch [300/10000], Loss: 0.6792
Epoch [400/10000], Loss: 0.6792
Epoch [500/10000], Loss: 0.6792
Epoch [600/10000], Loss: 0.6792
Epoch [700/10000], Loss: 0.6792
Epoch [800/10000], Loss: 0.6792
Epoch [900/10000], Loss: 0.6792
Epoch [1000/10000], Loss: 0.6792
Epoch [1100/10000], Loss: 0.6792
Epoch [1200/10000], Loss: 0.6792
Epoch [1300/10000], Loss: 0.6792
Epoch [1400/10000], Loss: 0.6792
Epoch [1500/10000], Loss: 0.6792
Epoch [1600/10000], Loss: 0.6792
Epoch [1700/10000], Loss: 0.6792
Epoch [1800/10000], Loss: 0.6792
Epoch [1900/10000], Loss: 0.6792
Epoch [2000/10000], Loss: 0.6792
Epoch [2100/10000], Loss: 0.6792
Epoch [2200/10000], Loss: 0.6792
Epoch [2300/10000], Loss: 0.6792
Epoch [2400/10000], Loss: 0.6792
Epoch [2500/10000], Loss: 0.6792
Epoch [2600/10000], Loss: 0.6792
Epoch [2700/10000], Loss: 0.6792
Epoch [2800/10000], Loss: 0.6792
Epoch [2900/10000], Loss: 0.6792
Epoch [3000/10000], Loss: 0.6792
Epoch [3100/10000],

# Evaluating the trained model

to get an output from a simple input

In [22]:

with torch.no_grad():
    # Now let's predict
    new_machine = torch.tensor([[10]], dtype=torch.float32)
    
    predicted_prob = model(new_machine)
    
    # make it binary...into true or false
    predicted_class = (predicted_prob > 0.5).float() 
    
    print(f"\nPredicted probability of working for 900 hours of usage: {predicted_prob.item():.4f}")
    print(f"Predicted class (0=Broken, 1=Working): {predicted_class.item()}")


Predicted probability of working for 900 hours of usage: 0.5833
Predicted class (0=Broken, 1=Working): 1.0
